<a href="https://colab.research.google.com/github/bryandaetz1/SB_County_COVID-19_Data/blob/master/SB_County_Public_Health_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [34]:
#specify URL/web page to scrape
url = 'https://publichealthsbc.org/status-reports/'

#create user-agent (Windows 10 with Google Chrome)
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '\
'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.03987.149 '\
'Safara/537.36'

headers = {'User-Agent': user_agent}

response = requests.get(url, headers = headers).text

In [35]:
#parse the HTML from the URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(response, 'html.parser')

In [36]:
#grabbing page title to verify that we were able to successfully get requests from webpage 
soup.title

<title>Status Reports – County of Santa Barbara</title>

In [37]:
#after inspecting the page, it looks like all the tables that contain data we want have id = 'stats'
tables = soup.find_all('table', {'id':'stats'})
len(tables)

8

In [38]:
#trying to create function to convert html table to pandas dataframe, had to break this into two functions

def TableToList(table):
  #getting contents of table and converting the text into a list
  raw = list(table.find_all('td'))
  text = []
  for i in raw:
    text.append(i.text)
  return text

def ListToDf(list, no_columns):
  # calculating number of rows for dataframe
  rows = int(len(list) / no_columns)

  # converting list to numpy array to reshape
  array = np.array(list)
  array = np.reshape(array, (rows, no_columns))

  #converting array to pandas dataframe
  df = pd.DataFrame(array[1:], columns = array[0])
  return df


In [39]:
table1 = TableToList(tables[0])
#table1

In [40]:
current_stats = ListToDf(table1, 6)
current_stats

,Geographic Area as of 7/2/2020,DailyCases,Total Confirmed Cases,Recovered by Region,Active Cases by Region,Number of Deaths
0,SOUTH COUNTY UNINCORPORATED AREA includes comm...,2,58,48,9,0
1,CITY OF SANTA BARBARA and the unincorporated a...,18,334,273,59,3
2,CITY OF GOLETA,2,66,50,14,2
3,COMMUNITY OF ISLA VISTA,0,17,12,5,0
4,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,0,47,40,6,1
5,SANTA YNEZ VALLEY including the Cities of Solv...,0,21,20,1,0
6,CITY OF LOMPOC and the communities of Mission ...,5,161,142,15,4
7,FEDERAL PRISON IN LOMPOC,0,994,991,0,3
8,CITY OF SANTA MARIA,69,1275,1041,220,15
9,COMMUNITY OF ORCUTT,3,81,71,9,0


In [41]:
table2 = TableToList(tables[1])
table2 = table2[1:]
#table2

In [42]:
cases_by_age = ListToDf(table2, 5)
cases_by_age

,Age Range,Daily,Community,Federal Prisonin Lompoc,Total(Community& Prison)
0,0 – 17,12,204,0,204
1,18 – 29,30,632,134,766
2,30 – 49,41,837,526,1363
3,50 – 69,25,465,302,767
4,70+,3,129,32,161
5,Age Suppressed,0,—,—,—


In [43]:
table3 = TableToList(tables[2])
table3 = table3[1:]
#table3

In [55]:
cases_by_sex = ListToDf(table3, 5)
cases_by_sex.columns.values[0] = 'Sex'
cases_by_sex

,Sex,Daily,Community,Federal Prisonin Lompoc,Total(Community& Prison)
0,Female,53,1076,2,1078
1,Male,56,1175,979,2154
2,Unknown,2,16,13,29
3,Pending,0,0,0,0


In [45]:
table4 = TableToList(tables[3])
#table4

In [46]:
testing_status = ListToDf(table4, 2)
testing_status

,Testing Status,Total
0,Positive,3261
1,Negative,40769
2,Inconclusive,134
3,Total Tests,44164


In [47]:
table5 = TableToList(tables[4])
#table5

In [48]:
high_risk = ListToDf(table5, 2)
high_risk

,High Risk,Total
0,Health Care Workers (HCW),157


In [49]:
table6 = TableToList(tables[5])
#table6

In [50]:
transmission_method = ListToDf(table6, 2)
transmission_method

,Transmission Method,Total
0,Community Close Contact Transmission,703
1,Persons Incarcerated at Federal Prison in Lomp...,994
2,Travel Transmission,12
3,Community Transmission,542
4,Under Investigation,1010


In [51]:
table7 = TableToList(tables[6])

#ran into an issue convering list to dataframe because website had missing values after 'Recovering in Home' and 'NO LONGER REPORTING', using a rough workaround for now to add "--" to the places where values were missing
table7.insert(13, '--')
table7.insert(15, '--')
#table7

In [52]:
recovery_status = ListToDf(table7, 4)
recovery_status

,RECOVERY STATUS,Community,Federal Prison in Lompoc,Total(Community & Prison)
0,Active Cases,374,0,374
1,Recovered Cases,1867,991,2858
2,Recovering at Home,--,NO LONGER REPORTING,--
3,Recovering in Hospital,—,—,64
4,Recovering in ICU,—,—,20
5,Pending Information,—,—,—


In [53]:
table8 = TableToList(tables[7])
#table8

In [54]:
ethnicity = ListToDf(table8, 5)
ethnicity

,RACE/ETHNICITY,Daily,Community,Federal Prison in Lompoc,Total(Community & Prison)
0,Hispanic/Latino,44,1145,377,1522
1,"White, Non-Hispanic",2,178,383,561
2,"Asian, Non-Hispanic",0,10,51,61
3,"Black/African American, Non-Hispanic",0,8,147,155
4,"American Indian/Native Alaskan, Non-Hispanic",1,5,28,33
5,"Native Hawaiian or Pacific Islander, Non-Hispanic",0,2,1,3
6,"Multiracial, Non-Hispanic",0,5,0,5
7,"Other, Non-Hispanic",1,15,0,15
8,Unknown Ethnicity,6,97,0,97
9,"Unknown Race, Non-Hispanic",2,13,0,13
